# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import csv
import logging
import os

import numpy as np
import pandas as pd
import pkg_resources

import azureml.core
from azureml.core import Dataset, Experiment, Workspace
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.pipeline.steps import AutoMLStep
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

from sklearn import datasets
from sklearn.model_selection import train_test_split

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize workspace from config

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Create Azure ML Experiment

Create and name the experiment

In [ ]:
# Choose a name for experiment
experiment_name = 'automl-heart-failure'
project_folder = './heartfailure'

experiment = Experiment(ws, experiment_name)

## Create or Attach an Auto ML Compute Cluster
Search for existing compute cluster; if not found, create one

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# If using a different existing cluster, enter the name in place of 'mlecscompute' below:
amlcompute_cluster_name = 'mlecscompute'

# Verify cluster existence: 
try: 
    compute_target = ComputeTarget(workspace = ws, name = amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Not found; creating new cluster.')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_v2', max_nodes=4, min_nodes=1)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output = True)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


Search for uploaded dataset in AzureML Studio; if not found, find it through the URL

In [ ]:
# Attempt to load the dataset from the Workspace, but otherwise, create from file
# dataset file located at 'https://raw.githubusercontent.com/RachelAnnDrury/MLECapstone/main/heart_failure_clinical_records_dataset.csv'
found = False
# key should be set to 'heartfailuredataset' 
key = 'heartfailuredataset'
# description_text should be set to 'heartfailuredataset' 
description_text = 'heartfailuredataset'

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]

if not found:
    datasetfile = 'https://raw.githubusercontent.com/RachelAnnDrury/MLECapstone/main/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(datasetfile)
    dataset = dataset.register(workspace = ws, name = key, description = description_text)

df = dataset.to_pandas_dataframe()
df.describe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# AutoML settings
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# AutoML config
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data = dataset,
                             label_column_name = "DEATH_EVENT",   
                             path = project_folder,
                             enable_early_stopping = True,
                             featurization = 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [12]:
# Submit experiment
automl_run = experiment.submit(config = automl_config, show_output = True)

        33   MinMaxScaler RandomForest                      0:00:56       0.8226    0.8529
        34   StandardScalerWrapper LightGBM                 0:00:40       0.8391    0.8529
        35                                                  0:00:27          nan    0.8529
        36                                                  0:00:17          nan    0.8529
        37                                                  0:00:20          nan    0.8529
        38    VotingEnsemble                                0:01:49       0.8797    0.8797
        39    StackEnsemble                                 0:01:55       0.8730    0.8797


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [14]:
import joblib

best_automl_run, fitted_automl_model = automl_run.get_output()
print(best_automl_run)
print(fitted_automl_model)

joblib.dump(fitted_automl_model, 'automlheartmodel')

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: automl-heart-failure,
Id: AutoML_5031ebf5-453b-47d8-b21e-a09dfb5c0976_38,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                random_state=None,
                                                                                                reg_alpha=0.3157894736842105,
                 

['automlheartmodel']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service